In [ ]:
# Project info
PROJECT_ID = 'marvin-voice'
GIT_OWNER = 'patrikkj'
GIT_REPO = 'marvin-models'
DATASET = 'dataset' # either 'dataset' or 'dataset_full'

# Download code from Git
!git clone -q https://github.com/{GIT_OWNER}/{GIT_REPO}.git

# Download and unpack dataset from GCS bucket
!gsutil -q cp gs://{PROJECT_ID}/{DATASET}.zip /tmp
!unzip -q /tmp/{DATASET}.zip -d /content/{GIT_REPO}/data/{DATASET}

# Make scripts accessible within notebook
import sys
sys.path.append(f"/content/{GIT_REPO}")

In [ ]:
!git -C marvin-models status